# Day 8: Treetop Tree House
https://adventofcode.com/2022/day/8

In [2]:
let lines: Vec<String> = std::fs::read_to_string("input/day08.txt")
    .expect("Could not read file")
    .lines()
    .map(|line| line.to_owned())
    .collect();

In [3]:
let example: Vec<String> = "30373
25512
65332
33549
35390".lines().map(|line| line.to_owned()).collect();

In [4]:
#[derive(Debug)]
struct Grid {
    width: usize,
    height: usize,
    trees: Vec<Vec<u8>>
}

In [5]:
impl Grid {
    fn parse(lines: &[String]) -> Self {
        let trees: Vec<Vec<u8>> = lines.iter().map(|line|
            line.chars()
                .map(|c| c as u8 - '0' as u8)
                .collect()
        ).collect();
        
        Self {
            height: trees.len(),
            width: trees.first().unwrap().len(),
            trees
        }
    }
    
    fn get_tree_height(self: &Self, (x, y): (usize, usize)) -> u8 {
        self.trees[y][x]
    }
}

## Part 1
Check if there is a path from each tree to an edge, where there are no trees of equal or greater height.

In [6]:
impl Grid {
    fn paths_towards_edge(self: &Self, (x, y): (usize, usize)) -> Vec<Vec<(usize, usize)>> {
        let left = (0..x).rev().map(|j| (j, y));
        let right = (x + 1..self.width).map(|j| (j, y));
        let top = (0..y).rev().map(|i| (x, i));
        let bottom = (y + 1..self.height).map(|i| (x, i));
        
        vec![left.collect(), right.collect(), top.collect(), bottom.collect()]
    }

    fn visible(self: &Self, (x, y): (usize, usize)) -> bool {
        let tree_height = self.get_tree_height((x, y));
        
        self.paths_towards_edge((x, y)).iter()
            .any(|path|
                path.iter().all(|other_tree| self.get_tree_height(*other_tree) < tree_height))
    }
}

In [7]:
fn part1(input: &[String]) -> usize {
    let grid = Grid::parse(&input);
    let grid_ref = &grid;
    
    (0..grid.height)
        .flat_map(|y| (0..grid.width).map(move |x| grid_ref.visible((x, y))))
        .filter(|visible| *visible)
        .count()
}

In [8]:
part1(&example)

21

In [9]:
part1(&lines)

1690

## Part 2
Count how many trees are visible from a tree in each of the four directions (top, bottom, left, right), i.e., how many trees there are up to and including the first one which has the same or greater height. Then calculate the product of these numbers.

In [10]:
impl Grid {
    fn scenic_score(self: &Self, pos: (usize, usize)) -> usize {
        let tree_height = self.get_tree_height(pos);

        self.paths_towards_edge(pos).iter()
            .map(|path| {
                // This solution is not quite functional. take_until could help, see references.
                let mut count = 0usize;
                for other_pos in path {
                    count += 1;
                    if self.get_tree_height(*other_pos) >= tree_height {
                        break;
                    }
                }
                
                count
            })
            .product()
    }
}

In [11]:
Grid::parse(&example).scenic_score((2, 3))

8

In [12]:
fn part2(input: &[String]) -> usize {
    let grid = Grid::parse(&input);
    let grid_ref = &grid;
    
    (0..grid.height)
        .flat_map(|y| (0..grid.width).map(move |x| grid_ref.scenic_score((x, y))))
        .max().unwrap()
}

In [13]:
part2(&example)

8

In [14]:
part2(&lines)

535680

## References
* Why do we need `move` and a reference to the grid in `fn part1` to make the nested `flat_map`/`map` structure compile? See
    * https://stackoverflow.com/a/70286326
    * https://stackoverflow.com/a/72442223
* `take_until` could be used to solve part 2 in a more functional way: https://docs.rs/take-until/latest/take_until/